# RNN 구현하기

In [1]:
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_size = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

inputs = np.random.random((timesteps, input_size)) # 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.

In [2]:
print(hidden_state_t) # 8의 크기를 가지는 은닉 상태. 현재는 초기 은닉 상태로 모든 차원이 0의 값을 가짐.

[0. 0. 0. 0. 0. 0. 0. 0.]


In [3]:
Wx = np.random.random((hidden_size, input_size))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

In [4]:
print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [5]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0) 
# 출력 시 값을 깔끔하게 해준다.

print(total_hidden_states) # (timesteps, output_dim)의 크기. 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.86786384 0.55225016 0.89544158 0.64733001 0.55266063 0.86706957
  0.95770841 0.82727334]
 [0.99988161 0.99851466 0.99905505 0.99787929 0.99983902 0.99940113
  0.99997714 0.99951076]
 [0.99996931 0.99976182 0.99969915 0.999763   0.99999262 0.99994526
  0.99999793 0.9999107 ]
 [0.99995606 0.99965693 0.99952808 0.99943458 0.99998627 0.99987407
  0.99999519 0.99979184]
 [0.99996728 0.99973203 0.99983378 0.9997923  0.99998552 0.99994919
  0.99999786 0.9998671 ]
 [0.99997036 0.99975866 0.99975835 0.99962485 0.99998168 0.99993708
  0.99999836 0.99988419]
 [0.99998221 0.99979366 0.99981634 0.9996871  0.99998658 0.99994579
  0.99999898 0.99993123]
 [0.99996059 0.99975642 0.99979224 0.99969141 0.99997548 0.99995004
  0.99999833 0.99985635]
 [0.99995809 0.99973128 0.999809   0.99964659 0.999967   0.99994236
  0.99999806 0.99981542]
 [0.99997636 0.99971788 0.99984186 0.99962147 0.99997696 0.99992205
  0.99999808 0.99985811]

# 파이토치 nn.RNN()

In [6]:
import torch
import torch.nn as nn

In [7]:
input_size = 5 # 입력의 크기
hidden_size = 8 # 은닉 상태의 크기

In [8]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

In [9]:
cell = nn.RNN(input_size, hidden_size, batch_first=True)

In [10]:
cell

RNN(5, 8, batch_first=True)

In [11]:
outputs, _status = cell(inputs)


In [12]:
print(outputs.shape) # 모든 time-step의 hidden_state


torch.Size([1, 10, 8])


In [13]:
print(_status.shape) # 최종 time-step의 hidden_state


torch.Size([1, 1, 8])


Deep RNN

In [14]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

In [15]:
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)


In [16]:
print(outputs.shape) # 모든 time-step의 hidden_state


torch.Size([1, 10, 8])


In [17]:
print(_status.shape) # (층의 개수, 배치 크기, 은닉 상태의 크기)


torch.Size([1, 1, 8])


양방향 RNN

In [18]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

In [19]:
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True, bidirectional = True)

In [20]:
outputs, _status = cell(inputs)


In [21]:
print(outputs.shape) # (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2)


torch.Size([1, 10, 16])


In [22]:
print(_status.shape) # (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)


torch.Size([4, 1, 8])


# LSTM

In [ ]:
nn.LSTM(input_dim, hidden_size, batch_fisrt=True)  
